# User Data Dictionary 

## Input

Reranked RAG Results from Cohere Rerank model see docs => [Cohere Rerank Documentation](https://docs.cohere.com/reference/rerank-1)

## Outcome

Produces a dataframe of data that suggests variable options based on a user input data dictionary in phase 1.

### Output Dataframe structure:

First 3 columns are related to a row of user data: 

user_study | user_var | user_label 

The next columns are specific to the LLM Output. Cohere will return the top 3 values for each variable and each option will contain the following columns:

| option_X_var | option_X_score | option_X_label | option_X_study

#### Conditional output scenarios

1) If one of the options returns a significantly high match and there are no other close competing options, then we will only return the first value
2) If there are no good options we return the options AND prompt our LLM to return an explantion of why the three options are faulty
3) If all/multiple options have a close significant match score we return all three options AND query a vector database on which variable's study most closely matches our current user submitted study (TBD)

| Column 1 | Column 2 | Column 3 |
| -------- | -------- | -------- |
| Cell 1A  | Cell 2A  | Cell 3A  |
| Cell 1B  | Cell 2B  | Cell 3B  |